# Create geographical data listing observatory locations

In [73]:
import numpy as np
import pandas as pd
import pickle

from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy.io.misc import yaml

Define some lists of names understood by `astropy.coordinates.EarthLocation`:

In [2]:
n_america = [
    'Apache Point',
    'Catalina Observatory',
    'Discovery Channel Telescope',
    'Kitt Peak',
    'Lick Observatory',
    'Lowell Observatory',
    'Mt Graham',
    'Palomar',
    'Very Large Array',
    'Whipple Observatory',
    'McDonald Observatory',
    'Sunspot',
    'Green Bank Telescope',
    'Observatorio Astronomico Nacional, San Pedro Martir',
    'Observatorio Astronomico Nacional, Tonantzintla',
    'Dominion Astrophysical Observatory',
    'Manastash Ridge Observatory',
    'Black Moshannon Observatory',
]

s_america = [
    'ALMA',
    'Cerro Paranal',
    'Cerro Tololo',
    'Gemini South',
    'La Silla Observatory',
    'Las Campanas Observatory',
    # 'Paranal Observatory',
    'National Observatory of Venezuela',
]

oz = [
    'Mt. Stromlo Observatory',
    'Murchison Widefield Array',
    'Siding Spring Observatory',
    'Anglo-Australian Observatory',
]

hawaii = [
    'Keck Observatory',
    'Subaru Telescope',
    'James Clerk Maxwell Telescope',
]

europe = [
    'greenwich',
    'Noto',
    'Mt. Ekar 182 cm. Telescope',
    'TUG',
    'Roque de los Muchachos',
    'Medicina Dish',
]

asia = [
    'Vainu Bappu Observatory',
    'Beijing XingLong Observatory',
]

africa = [
    'Southern African Large Telescope',
]

regions = { 
    'NorthAmerica': n_america,
    'SouthAmerica': s_america,
    'Australia': oz,
    'EuropeMed': europe,
    'Asia': asia,
    'Africa': africa
}

Some useful data-munging functions:

In [45]:
def get_locations(sites):
    earth_locs = {}
    for s in sites:
        earth_locs[s] = EarthLocation.of_site(s)
    return earth_locs

def make_locations_df(curr_region):
    sites = regions[curr_region]
    curr_locations = get_locations(sites)

    df = pd.DataFrame()
    df['name'] = sites
    df['lat'] = [loc.lat.value for name, loc in curr_locations.items()]
    df['lon'] = [loc.lon.value for name, loc in curr_locations.items()]
    df['region'] = [curr_region] * len(sites)
    
    return df, curr_locations

We want a Pandas dataframe in a format understood by MapBox, plus a dictionary containing EarthLocation objects:

In [46]:
df_list = []
locations = {}
for curr_region in regions:
    df_region, locs_region = make_locations_df(curr_region)
    df_list.append(df_region)
    locations.update(locs_region)
    
df = pd.concat(df_list, ignore_index=True)

Write the dataframe in csv format and the dictionary as yaml. The latter is maybe less efficient than pickle, but considered safer (pickle files can be hacked to execute arbitrary Python code).

In [71]:
df.to_csv('observatories.csv')

with open('obs.yaml', 'w') as file:
    td = yaml.dump(locations, file)

Make sure we can read the files in again!

In [77]:
df2 = pd.read_csv('observatories.csv')
df2[:3]

,Unnamed: 0,name,lat,lon,region
0,0,Apache Point,32.780000,-105.820000,NorthAmerica
1,1,Catalina Observatory,32.416667,-110.731667,NorthAmerica
2,2,Discovery Channel Telescope,34.744305,-111.422515,NorthAmerica


In [79]:
with open('obs.yaml', 'r') as file:
    td2 = yaml.load(file)
display(td2['Kitt Peak'])

<EarthLocation (-1994502.60430614, -5037538.54232911, 3358104.99690298) m>

Two examples of using the EarthLocation values, to get Alt/Az positions for an object then using astroplan to get sunset time:

In [86]:
current_time = Time.now()
obj_name = 'm31'
obj_coord = SkyCoord.from_name(obj_name)
aa_now = AltAz(location=td2['Kitt Peak'], obstime=current_time)
sky_coord = obj_coord.transform_to(aa_now)
display(f"{obj_name}: currently Alt = {sky_coord.alt:.2f}, Az = {sky_coord.az:.2f}")

'm31: currently Alt = 79.44 deg, Az = 25.23 deg'

In [92]:
try:
    from astroplan import Observer
    
    obs = Observer(location=td2['Kitt Peak'])
    sunset = obs.sun_set_time(current_time, which="next")
    print(f'Sunset times, ISO: {sunset.iso}, JD: {sunset.jd}')
except:
    print('sunset needs the astroplan package')

Sunset times, ISO: 2020-01-12 00:36:36.626, JD: 2458860.5254239123


In [49]:
import plotly.graph_objects as go
import plotly.express as px

default_size = 8

px.set_mapbox_access_token(open("/home/colin/.mapbox_token").read())
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name="name",
                  size_max=500, zoom=1) # color="region", 
fig.update_traces(marker=dict(size=default_size))

f = go.FigureWidget(fig)
# display(f.data)
scatter = f.data[0]

selection = 'test'

# create our callback function
def update_point(trace, points, selector):
    global selection
    selection = df.iloc[points.point_inds]
#     display(points.point_inds)
#     display(selection)
#     display(trace)
    f.update_layout(title=selection['name'].values[0])
#     c = ['green',] * len(trace.lat)
    s = [default_size,] * len(trace.lat)
    for i in points.point_inds:
        s[i] = default_size * 2
        with f.batch_update():
            scatter.marker.size = s

scatter.on_click(update_point)

f

FigureWidget({
    'data': [{'hoverlabel': {'namelength': 0},
              'hovertemplate': '<b>%{hovertext}<…

In [50]:
selection

,name,lat,lon,region
24,National Observatory of Venezuela,8.79,-70.866667,SouthAmerica
